In [1]:
# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#Resolvendo problema do chromeDrive
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# Pandas
import pandas as pd
# RegEx / Regular Expressions
import re
# Operating system
import os
import time

In [2]:
mann_pecas = 'http://cigamcatalogo.mannpecas.com.br/'
user = '91982496001920'
password = '9020'
notFound = pd.DataFrame()
notFound["Cod. Produto"] = ''
notFound['Nome'] = ''
notFound["Cod. Testado"] = ''
notFound['Motivo'] = ''
cwd = os.getcwd()

In [3]:
query = pd.read_csv('produtos_bot_yury_not_vv.txt', encoding = 'latin1', sep=',')
df_query = pd.DataFrame(query)
df_query.drop('IDX', axis=1, inplace=True)

In [4]:
def contains_number(string):
    return any(char.isdigit() for char in string)

In [5]:
def findOEM(text):
    oem = []
    if 'VV' in text:
        indice = text.find('VV')
        text = text.replace(text[indice:], '')
    if 'MCL' in text:
        cod_mcl = text.find('MCL')
        copy_text = text
        text = text[:cod_mcl]
        cod_mcl = copy_text[cod_mcl:]
        cod_mcl = cod_mcl.replace(' ', '')
        if contains_number(cod_mcl):
            if len(cod_mcl) > 5:
                oem.append(cod_mcl)
    if not contains_number(text):
        return oem
    num = re.search(r'\d+', text)
    indice = text.find(num.group())
    lastspace = text.rfind(' ', 0, indice)
    remover = [text[:lastspace], '.', '-']
    for rm in remover:
        text = text.replace(rm, '')
    text = text.replace(' ', '/')
    text = text.split('/')
    for txt in text:
        if contains_number(txt):
            if len(txt) >= 5:
                if not contains_number(txt[-1]):
                    txt = txt.replace(txt[-1], '')
                oem.append(txt)
    return oem

In [6]:
chrome_options = webdriver.ChromeOptions()
servico = Service(ChromeDriverManager().install())
#chrome_options.add_argument('headless')
nav = webdriver.Chrome(options=chrome_options, service=servico)
nav.maximize_window()
nav.get(str(mann_pecas))
nav.find_element(By.ID, 'usuario').send_keys(user)
nav.find_element(By.ID, 'senha').send_keys(password)
nav.find_element(By.ID, 'botao-login').click()
time.sleep(2)
with open(f'{cwd}/Mann Peças/prods.json', 'a') as arquivo:
            arquivo.write('[')
nav.get(str(mann_pecas + 'lst_produtos.php'))

In [13]:
def mann_pecas_search(cod, nome, index):
    produto = {}
    registrado = False
    nav.find_element(By.ID, 'pesquisa').send_keys(Keys.CONTROL + 'a')
    nav.find_element(By.ID, 'pesquisa').send_keys(Keys.BACKSPACE)
    nav.find_element(By.ID, 'pesquisa').send_keys(str(cod))
    nav.find_element(By.ID, 'pesquisa').send_keys(Keys.ENTER)
    element = nav.find_element(By.ID, 'dadosProduto')
    element = element.find_element(By.TAG_NAME, 'tbody')
    elements = element.find_elements(By.TAG_NAME, 'tr')
    keys = ['Preço Tabela :', 'Preço:', 'IPI', 'Base ST', 'Subst.Trib.:', 'Tipo.Op.:', 'Desconto:', 'Preço Final', 'Estoque:']
    for element in elements:
        if element == elements[-1]:
            img = element.find_element(By.TAG_NAME, 'a').get_attribute('href')
            produto.update({"Imagem: " : img})
        else:
            tds = element.find_elements(By.TAG_NAME, 'td')
            if len(tds) > 1:
                if tds[0].text not in keys:
                    try:
                        produto.update({tds[0].text : int(tds[1].text)})
                    except:
                        produto.update({tds[0].text : tds[1].text})
    produto.update({"Cod. Cotripal: " : index})
    for key in produto.keys():
        if cod == produto[key] and registrado == False:
            registrado = True
    if registrado:
        produto = str(produto)
        produto = produto.replace('"', '\"')
        produto = produto.replace("'", '"')
        with open(f'{cwd}/Mann Peças/prods.json', 'a') as arquivo:
            arquivo.write(produto)
            arquivo.write(',')
    else:
        notFound.loc[len(notFound)] = [index, nome, cod, 'não encontrado no site']

In [8]:
# time.sleep(2)
# mann_pecas_search('CQ27636','HASTE CAIXA PENEIRA CQ-276.36/Z-230.58', 1)
# mann_pecas_search('AE00101', 1)

In [9]:
with open(f'{cwd}/Mann Peças/prods.json', 'a') as arquivo:
            arquivo.write(']')

In [14]:
time.sleep(2)
produtos = dict(zip(df_query['ET0100_PRODUTO'], df_query['ET0100_NOME']))
for cod, nome in produtos.items():
    cods = findOEM(nome)
    if cods:
#         print(f'{cod} -> {nome} = {cods}')
        for oem in cods:
            mann_pecas_search(oem, nome, cods)
    else:
        notFound.loc[len(notFound)] = [cod, nome, '', 'não foi possível obter o código OEM'] 
#     for item in cods:
#         mann_pecas_search(item,nome, cod)

In [11]:
notFound.to_excel(f'{cwd}/Mann Peças/notFound.xlsx')